##2021-07-26 log
* y 값 설정해주는걸 좀 고심해서 해봐야 할 것 같다. 어떻게 할지? 아무래도 데이터 불러오는걸 따로 함수로 빼야할 것 같음.
* CNN feature extractor 수정이 필요함.
* 논문에서는 LSTM으로 했지만 굳이 LSTM으로 할 필요가 있나.. 하는 생각이 든다. RNN 으로 해도 될듯.

In [1]:
from google.colab import drive 
drive.mount('/content/gdrive/')


Mounted at /content/gdrive/


In [ ]:
!mkdir "/content/train_data/"
!unzip "/content/gdrive/MyDrive/GOTCHA/training_07_30.zip" -d /content/train_data

## pre-data
* updated : 2021-07-20


In [3]:
from keras import backend as K

from keras.layers import Input, Dense, Flatten, Activation, Dropout, Bidirectional, Permute, multiply
from keras.layers.recurrent import LSTM
from keras.models import Sequential, Model, load_model
from keras.applications.inception_v3 import InceptionV3

import glob

In [4]:
K.set_image_data_format('channels_first')

In [5]:
NUM_EPOCHS = 60
VERBOSE = 1
HIDDEN_UNITS = 3

In [6]:
def attention_block(inputs, time_steps):
    a = Permute((2, 1))(inputs)
    a = Dense(time_steps, activation='softmax')(a)
    a_probs = Permute((2, 1), name='attention_vec')(a) ## transpose 후 softmax를 빠져나와 다시 한번 더 transpose를 해줌으로서, 중요한 부분에만 가중치가 가도록 하는 역할.
    output_attention_mul = multiply([inputs, a_probs], name='attention_mul') ## input과 attention probability를 곱함.
    return output_attention_mul ## attention module 을 통과한 것을 내보냄.

In [7]:
import os 
from PIL import Image
import numpy as np

#/content/train_data/training/del/father/chr10_s.11780473_vp.11780468.png
def generate_batch(path, batch_size):
    total_x = glob.glob(path + "/**/father/*.png", recursive = True)
    print(len(total_x))
    num_batches = len(total_x) // batch_size
    while True:
      for batchIdx in range(0, num_batches):
        start = batchIdx * batch_size
        end = (batchIdx + 1 )* batch_size
        print(start, end)
        x_data = []
        y_data = []
        
        for k in range(start, end):
          info = total_x[k].split("/")
          x = np.array(Image.open(total_x[k]))
          temp = np.array([np.array(Image.open("/".join(info[0:5] + ["proband"] + [info[-1]]))), 
                          x ,
                          np.array(Image.open("/".join(info[0:5] + ["mother"] + [info[-1]])))])
          
          pos = np.zeros((64,1))
          pos[int((info[-1].split("_")[1]).split(".")[1]) - int((info[-1].split("_")[2]).split(".")[1])] = 1
          
          y_data.append(pos)
          x_data.append(temp)
        
        yield np.array(x_data), y_data


In [8]:
generate_batch("/content/train_data/training", 32)

<generator object generate_batch at 0x7f8bd563fbd0>

##BidirectionalLSTMClassifier
*updated : 2021-07-26


In [ ]:
class BidirectionalLSTMClassifier(object):
  def __init__(self, cnn_model_name, model_file = None):
    self.num_input_tokens = None
    self.nb_classes = None
    if model_file is None: self.model = None
    else:self.model = load_model(model_file)

    self.cnn_model_name = cnn_model_name.lower()

    def cnn_attention_lstm(self):
        inputs = Input(shape=(self.expected_frames, self.num_input_tokens,))
        attention_inputs = attention_block(inputs, self.expected_frames)
        lstm_out = Bidirectional(SimpleRNN(HIDDEN_UNITS, return_sequences=False))(attention_inputs) ## LSTM --> simple RNN
        x = Dense(512, activation='relu')(lstm_out)
        x = Dropout(0.5)(x)
        x = Dense(self.nb_classes, activation='softmax')(x)
        model = Model(input=[inputs], output=x)
        model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
        print(model.summary())
        return model

    def fit(self, input_path, output_path, data_set_name = '' , test_size = 0.3, attention = 'cnn_attention_lstm', do_feature_extraction = False):
      batch_size = 128

      cnn_model = InceptionV3(include_top=False, weights='imagenet')
      self.cnn_model_name = 'InceptionV3'
      feature_dir_name = data_set_name + '-' +  self.cnn_model_name + '-HiDimFeatures'

      labels = dict()

      feature_extractor = CNN_feature_extractor(self.cnn_model_name)
      x_samples, y_samples = feature_extractor.extract_cnn_features(input_path, 
                                                                    output_dir_path = feature_dir_name,
                                                                    model = cnn_model,
                                                                    data_set_name = data_set_name)
      
      if do_feature_extraction : return

      x_sample_0 = np.load(x_samples[0])
      print(x_sample_0.shape)

      self.num_input_tokens = x_sample_0.shape[1]

      frames_list = []

      for x in x_samples:
        frames = np.load(x).shape[0]
        frames_list.append(frames)
      
      self.expected_frames = int(np.mean(frames_list))
      print('expected frames : ', self.expected_frames)

      ########################################################
      for y in y_samples:
        if y not in labels:
          labels[y] = len(labels)
      
      for i in range(len(y_samples)):
        y_samples[i] = labels[y_samples[i]]

      self.nb_classes = len(labels)

      y_samples = np_utils.to_categorical(y_samples, self.nb_classes)

      ######################################################## 

      config = dict()
      config['labels'] = labels
      config['nb_classes'] = self.nb_classes
      config['num_input_tokens'] = self.num_input_tokens
      config['expected_frames'] = self.expected_frames

      np.save(os.path.join(output_path, 'config'), config)
      
      t1 = time.time()
      model = self.cnn_attention_lstm()
      csv_logger = CSVLogger(os.path.join(output_path, '{}_cnn_attention_lstm.log'.format(self.cnn_model_name)), append=True,
                                   separator=';')
      Xtrain, Xtest, Ytrain, Ytest = train_test_split(x_samples, y_samples, test_size=test_size,
                                                        random_state=None)
      
      train_gen = generate_batch(Xtrain, Ytrain, batch_size, self.expected_frames)
      test_gen = generate_batch(Xtest, Ytest, batch_size, self.expected_frames)

      train_num_batches = len(Xtrain) // batch_size
      test_num_batches = len(Xtest) // batch_size

      history = model.fit_generator(generator=train_gen, steps_per_epoch=train_num_batches,
                                    epochs=NUM_EPOCHS,
                                    verbose=1, validation_data=test_gen, validation_steps=test_num_batches,
                                    callbacks=[csv_logger])
      
      model_file_path = os.path.join(output_path, attention.replace('cnn', self.cnn_model_name) + '.h5')
      model.save(model_file_path)
      accu = history.history['val_acc'][-1]
      print('cnn-{}, attention-{}: accuracy-{}, time={}'.format(self.cnn_model_name, attention, accu, time.time()-t1))

      return accu


##CNN feature selector
* updated : 2021-07-26

In [ ]:
class CNN_Feature_extractor():
  def __init__ (self, cnn_model_name):
    self.cnn_model_name = cnn_model_name

  def extract_cnn_features_live():

  def cnn_features():

  def extract_cnn_features(self, input_data_dir_path, output_dir_path, model, data_set_name):
    y_samples = []
    x_samples = []

    for f in os.listdir(input_data_dir_path):
      file_path = input_data_dir_path + os.path.sep + f

